# Project Luther

## Looking at Steam data to predict several factors on user rating

In [1]:
#The basics
import numpy as np
import pandas as pd

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
#make sure to get a consistent table
import pickle
#need to pick apart strings
import re

In [5]:
test = ({'Name': ['Far Cry 5'],
                   'Price': [59.99],
                   'Percent off': [0],
                   'Rating percent': [71],
                   'User Ratings': [15500],
                   'Platforms': ['Windows, Mac'],
                   'Release Date': ['3/26/2018'],
                   'Publisher': ['Ubisoft'],
                   'Genre': ['Action, Adventure'],
                   'Number of DLC': [2],
                   'Total Cost of DLC': [38.98]})
df = pd.DataFrame(test)
df
#"""Use steam and steamdb.info. Add current players, peak players, average players, price change percentage, sale times, percent off."""

,Genre,Name,Number of DLC,Percent off,Platforms,Price,Publisher,Rating percent,Release Date,Total Cost of DLC,User Ratings
0,"Action, Adventure",Far Cry 5,2,0,"Windows, Mac",59.99,Ubisoft,71,3/26/2018,38.98,15500


# Let's try Selenium

In [44]:
"""driver = webdriver.Chrome(chromedriver)
driver.get('http://store.steampowered.com/search/?filter=topsellers')
time.sleep(1)"""

In [ ]:
"""title = driver.find_elements_by_xpath('//span[@class="title"]')
for c in title:
    print(c.text)"""

In [14]:
#Steamspy driver
driver = webdriver.Chrome(chromedriver)
driver.get('https://steamspy.com')

In [15]:
#scrapes four pages of steamspy for tables
#Data is scrapped on 4/21/2018
f = []
for i in range(4):
    i = driver.find_element_by_id('trendinggames')
    my_df = pd.read_html(driver.page_source)[0]
    i = i.text
    i = i.splitlines()
    i = i[5:]
    f.append(my_df)
    site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
    site.click()
    time.sleep(1)
    

In [165]:
#sanity check
pd.read_html(driver.page_source)[0]

,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners
0,1,Friday the 13th: Killer Puzzle,"Apr 12, 2018",Free,91% (95%),100000..200000
1,2,Radical Heights,"Apr 10, 2018",Free,12% (57%),500000..1000000
2,3,Farm Manager 2018,"Apr 6, 2018",$19.99,20% (65%),30000..100000
3,4,Minit,"Apr 3, 2018",$9.99,88% (94%/81%),0..30000
4,5,Welcome to the Game II,"Apr 3, 2018",$9.99,39% (78%),0..30000
5,6,PC Building Simulator,"Mar 27, 2018",$19.99,75% (91%),30000..100000
6,7,Band of Defenders,"Apr 5, 2018",$13.99,19% (64%),0..30000
7,8,Far Cry 5,"Mar 26, 2018",$59.99,27% (71%),500000..1000000
8,9,PixARK,"Mar 27, 2018",$24.99,20% (65%),100000..200000
9,10,Ni no Kuni II: Revenant Kingdom,"Mar 23, 2018",$59.99,53% (84%/81%),100000..200000


In [5]:
#Stacks the data and corrects the index issues
games_data = pd.concat(f)
games_data = games_data.reset_index()

NameError: name 'f' is not defined

In [18]:
#pickles the data for later
games_data.to_pickle('/Users/NickThomas/Project_Luther/data/games_data.pkl')

In [3]:
#pickle sucessfully loaded
with open('/Users/NickThomas/Project_Luther/data/games_data.pkl', 'rb') as pickle_file:
        games_data_loaded = pickle.load(pickle_file)
games_data_loaded

,index,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners
0,0,1,Friday the 13th: Killer Puzzle,"Apr 12, 2018",Free,90% (95%),"100,000 .. 200,000"
1,1,2,Radical Heights,"Apr 10, 2018",Free,11% (57%),"1,000,000 .. 2,000,000"
2,2,3,Farm Manager 2018,"Apr 6, 2018",$19.99,19% (65%),"20,000 .. 50,000"
3,3,4,Mirror,"Apr 19, 2018",$1.19,93% (96%),"200,000 .. 500,000"
4,4,5,Minit,"Apr 3, 2018",$9.99,87% (94%/81%),"0 .. 20,000"
5,5,6,Welcome to the Game II,"Apr 3, 2018",$9.99,38% (79%),"0 .. 20,000"
6,6,7,PC Building Simulator,"Mar 27, 2018",$19.99,73% (91%),"50,000 .. 100,000"
7,7,8,PixARK,"Mar 27, 2018",$24.99,19% (65%),"100,000 .. 200,000"
8,8,9,Far Cry 5,"Mar 26, 2018",$59.99,26% (71%),"500,000 .. 1,000,000"
9,9,10,PWND,"Apr 6, 2018",Free,28% (73%),"100,000 .. 200,000"


In [4]:
games_data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
index                                100 non-null int64
#                                    100 non-null int64
Game                                 100 non-null object
Release date                         100 non-null object
Price                                100 non-null object
Score rank(Userscore / Metascore)    100 non-null object
Owners                               100 non-null object
dtypes: int64(2), object(5)
memory usage: 5.5+ KB


In [13]:
games_data_loaded['Score rank(Userscore / Metascore)'].value_counts()

N/A (N/A)        64
93% (96%)         2
19% (65%)         2
69% (90%)         2
73% (91%)         2
50% (84%)         2
27% (72%)         1
50% (84%/81%)     1
N/A (N/A/77%)     1
22% (68%)         1
65% (89%)         1
20% (66%)         1
52% (85%)         1
N/A (N/A/65%)     1
87% (94%/81%)     1
26% (71%)         1
16% (62%)         1
47% (83%)         1
11% (57%)         1
21% (67%)         1
56% (86%)         1
90% (95%/89%)     1
18% (64%)         1
50% (84%/79%)     1
43% (81%)         1
38% (79%)         1
N/A (N/A/54%)     1
59% (87%)         1
90% (95%)         1
56% (86%/81%)     1
62% (88%)         1
28% (73%)         1
Name: Score rank(Userscore / Metascore), dtype: int64

In [17]:
games_data_loaded['#'].value_counts()

100    1
37     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
38     1
99     1
39     1
40     1
41     1
42     1
43     1
44     1
45     1
46     1
47     1
48     1
26     1
25     1
24     1
23     1
2      1
3      1
      ..
95     1
96     1
97     1
98     1
77     1
75     1
52     1
74     1
53     1
54     1
55     1
56     1
57     1
58     1
59     1
60     1
61     1
62     1
63     1
64     1
65     1
66     1
67     1
68     1
69     1
70     1
71     1
72     1
73     1
1      1
Name: #, Length: 100, dtype: int64

Uh-oh. Looks like I have a lot of missing values on the rating column. Is there anything workable?

In [16]:
mask = games_data_loaded['Score rank(Userscore / Metascore)'] == 'N/A (N/A)'
games_data_null = games_data_loaded[mask]
games_data_null

,index,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners
13,13,14,Fatal Twelve,"Mar 30, 2018",$19.99,N/A (N/A),"0 .. 20,000"
16,16,17,SINISTRY SILINIUM,"Apr 6, 2018",$4.99,N/A (N/A),"0 .. 20,000"
21,21,22,For The King,"Apr 19, 2018",$15.99,N/A (N/A),"100,000 .. 200,000"
23,23,24,AI Escort,"Apr 12, 2018",$1.99,N/A (N/A),"0 .. 20,000"
26,1,27,Cryptocurrency Clicker,"Apr 6, 2018",$1.99,N/A (N/A),"20,000 .. 50,000"
27,2,28,Find someone else,"Apr 6, 2018",Free,N/A (N/A),"0 .. 20,000"
28,3,29,Deployment,"Apr 10, 2018",$12.99,N/A (N/A),"0 .. 20,000"
29,4,30,Band of Defenders,"Apr 5, 2018",$13.99,N/A (N/A),"0 .. 20,000"
30,5,31,Healer's Quest,"Apr 18, 2018",$12.74,N/A (N/A),"0 .. 20,000"
31,6,32,Hakuoki: Edo Blossoms / /,"Mar 13, 2018",$29.99,N/A (N/A),"0 .. 20,000"


In [6]:
game_list = [games_data_loaded.Game.unique()]
game_list
driver = webdriver.Chrome(chromedriver)
driver.get('http://store.steampowered.com/search')
search = driver.find_element_by_xpath('//*[@id="term"]')
search.click()
search.send_keys("Friday the 13th: Killer Puzzle")
search.send_keys(Keys.RETURN)
rating = driver.find_element_by_class_name("search_review_summary")                                      
time.sleep(1)
rating.text

#Do not run until you're ready to get banned from steam
#"""for i in range(4):
#    i = driver.find_element_by_id('trendinggames')
#    my_df = pd.read_html(driver.page_source)[0]
#    i = i.text
#    i = i.splitlines()
#    i = i[5:]
#    f.append(my_df)
#    site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
#    site.click()
#    time.sleep(1)"""

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.12.6 x86_64)


In [15]:
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []

In [334]:
rating = driver.find_element_by_xpath('//*[@id="game_highlights"]/div[1]/div/div[3]/div/div[1]')
a = rating.text
#genre = driver.find_element_by_xpath('//*[@id="responsive_apppage_details_left_ctn"]/div[2]/div/div/div[1]/a[1]')
#b = genre.text
cost_dlc = driver.find_element_by_xpath('//*[@id="dlc_purchase_action"]/div[1]')
c = cost_dlc.text
#c = np.nan
#publisher = driver.find_element_by_xpath('//*[@id="responsive_apppage_details_left_ctn"]/div[2]/div/div/div[1]/a[5]')
#d = publisher.text
#developer = driver.find_element_by_xpath('//*[@id="responsive_apppage_details_left_ctn"]/div[2]/div/div/div[1]/a[6]')
#e = developer.text
f = re.compile(r'\d+\%')
g = re.findall(f, str(a))
h = re.compile(r'(?<=the\s).*(?=\suser)')
j = re.findall(h, str(a))
print(a)
print(c)
print(g)
print(j)

ALL REVIEWS:
3 user reviews - Need more user reviews to generate a score
nan
nan
3


In [335]:
#list1.append(a)
#list2.append(b)
list3.append(c)
#list4.append(d)
#list5.append(e)
list6.append(g)
list7.append(j)
print(list3)
print(list6)
print(list7)

['$18.89', '$14.99', '$12.95', '$2.38', nan, nan, nan, nan, '$39.98', nan, '$5.99', '$19.99', '$0.99', '$24.98', nan, nan, nan, nan, nan, nan, nan, '$4.99', nan, nan, nan, nan, nan, nan, nan, nan, '$6.99', '$12.98', nan, nan, nan, '$14.99', nan, '$0.49', nan, nan, '$24.79', nan, '$48.98', '$23.92', '$44.96', nan, nan, nan, nan, '$19.98', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '$23.90', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '$23.92', '$27.96', nan, nan, '$103.95', nan, nan, nan]
[['95%'], ['57%'], ['62%'], ['96%'], ['94%'], ['79%'], ['91%'], ['65%'], ['70%'], ['73%'], ['81%'], ['84%'], ['66%'], ['100%'], ['88%'], ['95%'], ['67%'], ['83%'], ['87%'], ['84%'], ['90%'], ['84%'], ['71%'], ['72%'], ['89%'], ['33%'], ['42%'], ['87%'], ['98%'], ['63%'], ['86%'], ['96%'], ['71%'], ['65%'], ['51%'], ['71%'], ['76%'], ['100%'], ['31%'], ['76%'], ['30%'], ['98%'], ['64%'],

In [342]:
dlc_cost = np.asarray(list3)
user_rating = np.asarray(list6)
number_of_reviews = np.asarray(list7)

In [343]:
form = {'dlc_cost': dlc_cost, 'user_rating': user_rating, 'number_of_reviews': number_of_reviews}
add_df = pd.DataFrame(form)
add_df.head()

,dlc_cost,number_of_reviews,user_rating
0,$18.89,"[1,184]",[95%]
1,$14.99,"[7,365]",[57%]
2,$12.95,[276],[62%]
3,$2.38,[982],[96%]
4,nan,[221],[94%]


In [344]:
add_df.to_pickle('/Users/NickThomas/Project_Luther/data/add_df.pkl')

In [345]:
with open('/Users/NickThomas/Project_Luther/data/add_df.pkl', 'rb') as pickle_file:
        add_df = pickle.load(pickle_file)
add_df

,dlc_cost,number_of_reviews,user_rating
0,$18.89,"[1,184]",[95%]
1,$14.99,"[7,365]",[57%]
2,$12.95,[276],[62%]
3,$2.38,[982],[96%]
4,nan,[221],[94%]
5,nan,[389],[79%]
6,nan,"[1,983]",[91%]
7,nan,"[2,490]",[65%]
8,$39.98,"[16,281]",[70%]
9,nan,[564],[73%]


In [107]:
game_list

[array(['Friday the 13th: Killer Puzzle', 'Radical Heights',
        'Farm Manager 2018', 'Mirror', 'Minit', 'Welcome to the Game II',
        'PC Building Simulator', 'PixARK', 'Far Cry 5', 'PWND', 'Jalopy',
        'Ni no Kuni II: Revenant Kingdom',
        'Golfing Over It with Alva Majo', 'Fatal Twelve',
        'Lobotomy Corporation | Monster Management Simulation',
        'The Curse of Monkey Island', 'SINISTRY SILINIUM',
        'Devil May Cry HD Collection', 'Sword With Sauce',
        'Darwin Project', 'Train Valley 2', 'For The King',
        "Otaku's Fantasy 2", 'AI Escort', 'Maelstrom', 'Extinction',
        'Cryptocurrency Clicker', 'Find someone else', 'Deployment',
        'Band of Defenders', "Healer's Quest", 'Hakuoki: Edo Blossoms / /',
        'Broken Ground', 'blue.', 'Achievement Idler: Black',
        'Dungeon Of Zaar - Open Beta', 'TT Isle of Man',
        'Road Scars: Origins', 'RUSSIA HORROR 20!8', 'Amorous',
        'Ys VIII: Lacrimosa of DANA / VIII-Lacrimos

In [346]:
games_data_loaded.join(add_df)

,index,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,dlc_cost,number_of_reviews,user_rating
0,0,1,Friday the 13th: Killer Puzzle,"Apr 12, 2018",Free,90% (95%),"100,000 .. 200,000",$18.89,"[1,184]",[95%]
1,1,2,Radical Heights,"Apr 10, 2018",Free,11% (57%),"1,000,000 .. 2,000,000",$14.99,"[7,365]",[57%]
2,2,3,Farm Manager 2018,"Apr 6, 2018",$19.99,19% (65%),"20,000 .. 50,000",$12.95,[276],[62%]
3,3,4,Mirror,"Apr 19, 2018",$1.19,93% (96%),"200,000 .. 500,000",$2.38,[982],[96%]
4,4,5,Minit,"Apr 3, 2018",$9.99,87% (94%/81%),"0 .. 20,000",nan,[221],[94%]
5,5,6,Welcome to the Game II,"Apr 3, 2018",$9.99,38% (79%),"0 .. 20,000",nan,[389],[79%]
6,6,7,PC Building Simulator,"Mar 27, 2018",$19.99,73% (91%),"50,000 .. 100,000",nan,"[1,983]",[91%]
7,7,8,PixARK,"Mar 27, 2018",$24.99,19% (65%),"100,000 .. 200,000",nan,"[2,490]",[65%]
8,8,9,Far Cry 5,"Mar 26, 2018",$59.99,26% (71%),"500,000 .. 1,000,000",$39.98,"[16,281]",[70%]
9,9,10,PWND,"Apr 6, 2018",Free,28% (73%),"100,000 .. 200,000",nan,[564],[73%]


In [79]:
game_rating
#Use regex '\d+\%' for review and use '(?<=the\s).*(?=\suser)' for number of users
#Also add for publisher, developer, and genre

array(['RECENT REVIEWS:\nOverwhelmingly Positive - 97% of the 832 user reviews in the last 30 days are positive.',
       'ALL REVIEWS:\nVery Positive - 94% of the 217 user reviews for this game are positive.'],
      dtype='<U103')

In [70]:

table = driver.find_element_by_id('trendinggames')
table_text = table.text
table_text_split = table_text.splitlines()
table_text_split = table_text_split[3:]
site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
site.click()

['# GAME RELEASE DATE',
 'PRICE',
 '',
 '1 Friday the 13th: Killer Puzzle Apr 12, 2018 Free',
 '2 Radical Heights Apr 10, 2018 Free',
 '3 Farm Manager 2018 Apr 6, 2018 $19.99',
 '4 Minit Apr 3, 2018 $9.99',
 '5 Welcome to the Game II Apr 3, 2018 $9.99',
 '6 PC Building Simulator Mar 27, 2018 $19.99',
 '7 Band of Defenders Apr 5, 2018 $13.99',
 '8 Far Cry 5 Mar 26, 2018 $59.99',
 '9 PixARK Mar 27, 2018 $24.99',
 '10 Ni no Kuni II: Revenant Kingdom Mar 23, 2018 $59.99',
 '11 Jalopy Mar 28, 2018 $14.99',
 '12 PWND Apr 6, 2018 Free',
 '13 Shroud of the Avatar: Forsaken Virtues Mar 27, 2018 $39.99',
 '14 while True: learn() Mar 28, 2018 $6.99',
 '15 Ghost of a Tale Mar 13, 2018 $24.99',
 '16 Lobotomy Corporation | Monster Management Simulation Apr 9, 2018 $24.99',
 '17 The Elder Scrolls V: Skyrim VR Apr 2, 2018 $59.99',
 '18 Devil May Cry HD Collection Mar 13, 2018 $29.99',
 '19 SYNTHETIK Mar 15, 2018 $14.99',
 '20 Darwin Project Mar 9, 2018 $14.99',
 '21 SINISTRY SILINIUM Apr 6, 2018 $4.99

In [74]:
site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
site.click()
table2 = driver.find_element_by_id('trendinggames')
table_text2 = table2.text
table_text_split2 = table_text2.splitlines()
table_text_split2 = table_text_split2[3:]

['# GAME RELEASE DATE',
 'PRICE',
 '',
 '26 Winions: Mana Champions Apr 12, 2018 Free',
 '27 Iron League Mar 14, 2018 Free',
 '28 AI Escort Apr 12, 2018 $1.99',
 '29 Atelier Lydie & Suelle ~The Alchemists and the Mysterious Paintings~ Mar 27, 2018 $59.99',
 "30 Otaku's Fantasy 2 Mar 30, 2018 $1.99",
 '31 Maelstrom Apr 11, 2018 $19.99',
 '32 KOEWOTAYORINI / Mar 26, 2018 Free',
 '33 Horizon Source() Mar 20, 2018 Free',
 '34 Moe Jigsaw Mar 23, 2018 $3.99',
 '35 World Warfare Apr 3, 2018 Free',
 '36 How To Cope With Boredom and Loneliness Apr 2, 2018 Free',
 '37 Clawface Mar 13, 2018 Free',
 '38 Project: Gorgon Mar 12, 2018 $39.99',
 '39 Will To Live Online Apr 5, 2018 $14.99',
 "40 Downtown Casino: Texas Hold'em Poker Apr 4, 2018 Free",
 '41 SAO Utils: Beta Mar 14, 2018 $3.99',
 '42 Solitaire Knights Apr 5, 2018 Free',
 '43 Road Scars: Origins Apr 3, 2018 $0.49',
 '44 Achievement Idler: Black Apr 13, 2018 $0.59',
 "45 Marie's Room Apr 13, 2018 Free",
 '46 Hero Academy 2 Apr 17, 2018 $19.9

In [98]:
site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
site.click()
table3 = driver.find_element_by_id('trendinggames')
table_text3 = table3.text
table_text_split3 = table_text3.splitlines()
table_text_split3 = table_text_split3[3:]
table_text_split3

['51 RUSSIA HORROR 20!8 Mar 19, 2018 $0.99',
 '52 Dark Rose Valkyrie Apr 10, 2018 $39.99',
 '53 AirMech Strike Mar 30, 2018 Free',
 '54 My Free Farm 2 Apr 3, 2018 Free',
 '55 Prismata Mar 8, 2018 $24.99',
 '56 EARTHLOCK Mar 8, 2018 $29.99',
 '57 Revenge of the Headless Apr 3, 2018 $3.99',
 '58 RUSSIAN PEACE DUCK : TAKE MY NALOGI Apr 9, 2018 $0.99',
 '59 Masters of Anima Apr 10, 2018 $19.99',
 '60 What is love? Mar 20, 2018 $3.99',
 '61 Tower of Time Apr 12, 2018 $24.99',
 '62 Dead In Vinland Apr 12, 2018 $19.99',
 '63 Project AURA Apr 17, 2018 $19.99',
 '64 Mirror Apr 19, 2018 $1.19',
 '65 Super Daryl Deluxe Apr 10, 2018 $19.99',
 '66 Double Kick Heroes Apr 11, 2018 $14.99',
 '67 Sisters In Hotel: Episode 2 Apr 2, 2018 $0.99',
 '68 Foreveracers Mar 29, 2018 $5.99',
 '69 Next 3 Apr 3, 2018 $0.99',
 '70 War Robots Apr 4, 2018 Free',
 '71 Broken Ground Apr 12, 2018 Free',
 '72 Bounce Apr 2, 2018 $0.99',
 '73 Knights Hunt Mar 20, 2018 $4.99',
 '74 Idle Adventure Apr 13, 2018 Free',
 '75 Th

In [138]:
table_text 

"# GAME RELEASE DATE\nPRICE\n\n1 Friday the 13th: Killer Puzzle Apr 12, 2018 Free\n2 Radical Heights Apr 10, 2018 Free\n3 Farm Manager 2018 Apr 6, 2018 $19.99\n4 Minit Apr 3, 2018 $9.99\n5 Welcome to the Game II Apr 3, 2018 $9.99\n6 PC Building Simulator Mar 27, 2018 $19.99\n7 Band of Defenders Apr 5, 2018 $13.99\n8 Far Cry 5 Mar 26, 2018 $59.99\n9 PixARK Mar 27, 2018 $24.99\n10 Ni no Kuni II: Revenant Kingdom Mar 23, 2018 $59.99\n11 Jalopy Mar 28, 2018 $14.99\n12 PWND Apr 6, 2018 Free\n13 Shroud of the Avatar: Forsaken Virtues Mar 27, 2018 $39.99\n14 while True: learn() Mar 28, 2018 $6.99\n15 Ghost of a Tale Mar 13, 2018 $24.99\n16 Lobotomy Corporation | Monster Management Simulation Apr 9, 2018 $24.99\n17 The Elder Scrolls V: Skyrim VR Apr 2, 2018 $59.99\n18 Devil May Cry HD Collection Mar 13, 2018 $29.99\n19 SYNTHETIK Mar 15, 2018 $14.99\n20 Darwin Project Mar 9, 2018 $14.99\n21 SINISTRY SILINIUM Apr 6, 2018 $4.99\n22 Spellsworn Mar 13, 2018 Free\n23 THE DAY Online Apr 3, 2018 Free\

In [41]:
def find_page(title):
    p = []
    for i in title:
        #driver.get('https://www.google.com/')
        time.sleep(1)
        location = driver.find_element_by_xpath('//*[@id="lst-ib"]')
        time.sleep(1)
        location.click()
        time.sleep(1)
        location.send_keys("steam spy")
        time.sleep(1)
        location.send_keys(Keys.RETURN)
        site = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/h3/a')
        site.click()
        time.sleep(1)
        score = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div/p[1]/strong[8]')
        driver.get('https://www.google.com/')
        s = score.text
        soup = BeautifulSoup(s, "lxml")
        p = soup.find_all('strong')
    return p

In [37]:
prices = driver.find_elements_by_xpath('//div[@class="col search_price  responsive_secondrow"]')
for i in prices:
    print(i.text)
discount_prices = driver.find_elements_by_xpath('//div[@class="col search_price discounted responsive_secondrow"]')
for b in discount_prices:
    print(b.text)
pos_rating = driver.find_elements_by_xpath('//*[@id="search_result_container"]/div[2]/a[2]/div[2]/div[3]/span')
for r in pos_rating:
    print(r.text)
mixed_rating = driver.find_elements_by_xpath('//span[@class="search_review_summary mixed"]')
for g in mixed_rating:
    print(g.text)
neg_rating = driver.find_elements_by_xpath('//span[@class="search_review_summary negative"]')
for v in neg_rating:
    print(v.text)

$59.99
$29.99
$39.99
$9.99
$19.99
$29.99
$59.99
$44.99
$59.99
$59.99
$14.99
$59.99
$23.99
$24.99
$8.49
$19.99
$15.99
$39.99
$19.99
$48.96
$20.30
$17.99
$7.19
$49.99
$24.99
$9.99
$7.49
$59.99
$19.79
$39.99
$21.99
$39.99
$21.99
$59.99
$41.99
$24.99
$18.74
$29.99
$14.99
Far Cry® 5
Dying Light Enhanced Edition
PLAYERUNKNOWN'S BATTLEGROUNDS
Darkest Dungeon®
For The King
BATTLETECH
Black Desert Online
The Elder Scrolls V: Skyrim Special Edition
Rocket League®
Warhammer: Vermintide 2
Darkest Dungeon®: Ancestral Edition
The Elder Scrolls V: Skyrim VR
Salt and Sanctuary
The Surge: Complete Edition
Divinity: Original Sin 2
Darkest Dungeon®: The Crimson Court
Rise of the Tomb Raider™
Arizona Sunshine
Arizona Sunshine
Total War: WARHAMMER II
Fallout 4 VR
Grand Theft Auto V
SUPERHOT VR
Rick and Morty: Virtual Rick-ality
Tom Clancy's Rainbow Six® Siege


























In [42]:
p = find_page(title)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="lst-ib"]"}
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.12.6 x86_64)


In [40]:
score = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div/p[1]/strong[8]')
score.text

'Price:'

In [17]:
p

<selenium.webdriver.remote.webelement.WebElement (session="fbbb69071316515f32eeebfa684ac910", element="0.11875028831938028-1")>

In [123]:
price

[]

In [63]:
url2 = 'https://steamspy.com/'

response = requests.get(url2)
print(response)

<Response [200]>


In [64]:
page = response.text

In [65]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta charset="utf-8"/>
  <title>
   SteamSpy - All the data and stats about Steam games
  </title>
  <meta content="Games sales" property="og:title"/>
  <meta content="SteamSpy - All the data about Steam games" property="og:site_name"/>
  <meta content="Steam Spy automatically gathers data from Steam user profiles, analyzes it and presents in simple, yet beautiful, manner. Steam Spy is designed to be helpful for indie developers, journalists, students and all parties interested in PC gaming and its current state of affairs." property="og:description"/>
  <meta content="en_US" property="og:locale"/>
  <meta content="https://www.facebook.com/galyonkin" property="article:author"/>
  <meta content="http://steamspy.com/assets/img/mascot.png" property="og:image"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <link href="/p

In [69]:
soup.find(class_ = "t1024")

<th class="t1024"><div data-toggle="tooltip" title="How many people own this game on Steam. Might be vastly different from sales.">Owners</div></th>

In [166]:
games_data

,index,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners
0,0,1,Friday the 13th: Killer Puzzle,"Apr 12, 2018",Free,91% (95%),100000..200000
1,1,2,Radical Heights,"Apr 10, 2018",Free,12% (57%),500000..1000000
2,2,3,Farm Manager 2018,"Apr 6, 2018",$19.99,20% (65%),30000..100000
3,3,4,Minit,"Apr 3, 2018",$9.99,88% (94%/81%),0..30000
4,4,5,Welcome to the Game II,"Apr 3, 2018",$9.99,39% (78%),0..30000
5,5,6,PC Building Simulator,"Mar 27, 2018",$19.99,75% (91%),30000..100000
6,6,7,Band of Defenders,"Apr 5, 2018",$13.99,19% (64%),0..30000
7,7,8,Far Cry 5,"Mar 26, 2018",$59.99,27% (71%),500000..1000000
8,8,9,PixARK,"Mar 27, 2018",$24.99,20% (65%),100000..200000
9,9,10,Ni no Kuni II: Revenant Kingdom,"Mar 23, 2018",$59.99,53% (84%/81%),100000..200000
